<a href="https://colab.research.google.com/github/chyj0805/Project_2/blob/main/Fetures/Vantage_API_data_fetches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv

# Load the environment variable file
load_dotenv()

# Get the API key
api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

In [ ]:
import requests
import pandas as pd
import os

def get_stock_historical_data(api_key, stock_symbol):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": stock_symbol,
        "apikey": api_key,
        "outputsize": "full",
        "datatype": "json"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    print(data)  # Debug line

    if "Time Series (Daily)" not in data:
        print("Error fetching data. Check your API key and stock symbol.")
        print(data)
        return

    time_series = data["Time Series (Daily)"]
    data_list = []

    for date, details in time_series.items():
        row = {
            "Date": date,
            "Open": details["1. open"],
            "High": details["2. high"],
            "Low": details["3. low"],
            "Close": details["4. close"],
            "Volume": details["5. volume"]
        }
        data_list.append(row)

    df = pd.DataFrame(data_list)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")

    csv_filename = f"{stock_symbol}_historical_data.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

stock_symbol = "IYE"
get_stock_historical_data(api_key, stock_symbol)

In [ ]:
def get_MACD_data(api_key, stock_symbol, interval="daily", series_type="close"):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "MACD",
        "symbol": stock_symbol,
        "interval": interval,
        "series_type": series_type,
        "apikey": api_key,
        "datatype": "json"
    }

    # Make the API request and fetch the data
    response = requests.get(base_url, params=params)
    data = response.json()

    # Debug line to print the data
    print(data)

    # Check if the MACD data is present in the response
    if "Technical Analysis: MACD" not in data:
        print("Error fetching MACD data. Check your API key and stock symbol.")
        print(data)
        return

    # Extract and process the MACD data
    macd_series = data["Technical Analysis: MACD"]
    macd_list = []
    for date, details in macd_series.items():
        row = {
            "Date": date,
            "MACD_Signal": details["MACD_Signal"],
            "MACD_Hist": details["MACD_Hist"],
            "MACD": details["MACD"]
        }
        macd_list.append(row)

    # Convert the list of MACD data to a DataFrame
    df = pd.DataFrame(macd_list)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values(by="Date")

    # Save the MACD data to a CSV file
    csv_filename = f"{stock_symbol}_MACD_data.csv"
    df.to_csv(csv_filename, index=False)
    print(f"MACD data saved to {csv_filename}")

# Check if the API key is set in the environment
api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
if api_key is None:
    raise ValueError("Alpha Vantage API key is not set in the environment.")

# Define the stock symbol for which to fetch the MACD data
stock_symbol = "IYE"

# Call the function to fetch and save the MACD data
get_MACD_data(api_key, stock_symbol)